In [1]:
import pandas as pd
import numpy as np

Upload raw Hematopoietic Cell Transplantation (**HCT**) data extracted from *2023 Annual Report* issued by National Transplantation Center on GITMO data:

In [2]:
file_path = '../data_raw/hct_raw_data.csv'
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       29 non-null     object
 1   1       247 non-null    object
 2   2       149 non-null    object
 3   3       149 non-null    object
 4   4       149 non-null    object
 5   5       148 non-null    object
 6   6       155 non-null    object
dtypes: object(7)
memory usage: 14.7+ KB


Data cleaning steps aim at having:
- all tranplant program names in one line vs. their split over more than one line in the original report,
- the number and type of HCTs performed at each transplant program in 2023,
- the right `dtype` in each column.

In [3]:
df = df.fillna('') # reproduce the same condition as from .pdf
df.head(10)

,0,1,2,3,4,5,6
0,Tabella G8- Numero trapianti autologhi e allog...,,,,,,
1,,,,,,Trapianti allogenici / Tipologia donatore,
2,Regione,Programmi Trapianto,Trapianti,Trapianti,Familiari,Non familiari,Familiari
3,,,autologhi,allogenici tot,compatibili,,parzialmente
4,,,,,,,compatibili
5,Abruzzo,248 - UOSD Terapia Intensiva Ematologica,59,32,8,10,14
6,,-Dipartimento Oncologico/Ematologico - Ospedale,,,,,
7,,Civile Santo Spirito -Pescara,,,,,
8,Basilicata,185 - UO Ematologia e Trapianto di Cellule Sta...,9,0,0,0,0
9,,-IRCCS/CROB Centro di Riferimento Oncologico,,,,,


The inspection of the above `df` DataFrame shows that:
- program names are all in column `1` that can be selected as `df.iloc[:, 1]`
- nr of autologous and allogeneic transplations are in coulmns `2` and `3`, respectively, accessible as `df.iloc[:, 2]` and `df.iloc[:, 3]` 
- **note** that the number of tranplants per program appear in the same row where the tranplant program name begins identified by its 3-digit code; this occurrence will be leveraged to construct the tranplant name all in one row starting from where `df.iloc[:, 2] != 0` (autologous tranplants are performed by all programs, while this is not the case for allo-transplants)
- columns `4`, `5`, and `6` contain the type of allo-transplant and the type of donor

In [4]:
# Step 1: Identify rows where a new program name starts using df.iloc[:, 2]
# This creates a boolean Series: True if the string in df.iloc[:,2] is all digits, False otherwise
# This is the key to identifying the beginning of a new program name block.
is_start_of_program = df.iloc[:, 2].str.isdigit()

In [5]:
# Step 2: Create a unique 'program_id' for each program block
# cumsum() is used here. It increments the ID every time a 'True' value appears in 'is_start_of_program'.
# This assigns a unique ID to the *first* row of each program block.
df['program_id'] = is_start_of_program.cumsum()

In [6]:
# Step 3: Forward fill 'program_id' to propagate it to all rows belonging to the same program
# This ensures that all fragmented parts of a single program name (which are in subsequent rows)
# get the same 'program_id' as their starting row.
df['program_id'] = df['program_id'].ffill()

In [7]:
# Step 4: Group by 'program_id' and concatenate the names from df.iloc[:, 1]
# We group all rows that share the same 'program_id'.
# Then, for each group, we take the strings from df.iloc[:, 1] (the actual name parts),
# convert them to string type (just in case), and join them with a space.
full_program_names_series = df.groupby('program_id')[df.columns[1]].apply(lambda x: ' '.join(x.astype(str)))

In [8]:
# Convert the resulting Series to a DataFrame and rename the column for clarity
full_program_names_df = full_program_names_series.reset_index(name='Full Program Name')

In [9]:
full_program_names_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   program_id         137 non-null    int32 
 1   Full Program Name  137 non-null    object
dtypes: int32(1), object(1)
memory usage: 1.7+ KB


In [10]:
# Step 5: Merge this back to the original DataFrame (optional, but often useful)
# Typically, you'd want to display the full program name alongside the original data
# for the *starting row* of each program.

# Filter the original DataFrame to keep only the rows where a program officially starts.
# These are the rows that initially had 'is_start_of_program' as True.
result_df = df[is_start_of_program].copy()

In [11]:
# Merge the 'Full Program Name' back into these starting rows, using 'program_id' as the key.
result_df = pd.merge(result_df, full_program_names_df, on='program_id', how='left')

# Drop any temporary columns created during the process if they are not needed in the final output.
# 'program_id' is no longer needed after the merge if you only want the final aggregated rows.
result_df.drop(columns=['program_id'], inplace=True)

In [12]:
result_df.head()

,0,1,2,3,4,5,6,Full Program Name
0,Abruzzo,248 - UOSD Terapia Intensiva Ematologica,59,32,8,10,14,248 - UOSD Terapia Intensiva Ematologica -Dipa...
1,Basilicata,185 - UO Ematologia e Trapianto di Cellule Sta...,9,0,0,0,0,185 - UO Ematologia e Trapianto di Cellule Sta...
2,,861 - Unità Operativa di Ematologia - Ospedale S.,12,0,0,0,0,861 - Unità Operativa di Ematologia - Ospedale...
3,Calabria,587 - Presidio Morelli CTMO Centro Unico Regio...,111,41,15,17,9,587 - Presidio Morelli CTMO Centro Unico Regio...
4,Campania,191 - UOC Ematologia - ASL Salerno -Pagani,20,0,0,0,0,191 - UOC Ematologia - ASL Salerno -Pagani


In [13]:
# Forward fill column '0' to propagate it to all rows belonging to the same Region
result_df['0'] = result_df['0'].replace('', np.nan)
result_df['0'] = result_df['0'].ffill()

In [14]:
# Print the final DataFrame with the reconstructed program names
result_df.head()

,0,1,2,3,4,5,6,Full Program Name
0,Abruzzo,248 - UOSD Terapia Intensiva Ematologica,59,32,8,10,14,248 - UOSD Terapia Intensiva Ematologica -Dipa...
1,Basilicata,185 - UO Ematologia e Trapianto di Cellule Sta...,9,0,0,0,0,185 - UO Ematologia e Trapianto di Cellule Sta...
2,Basilicata,861 - Unità Operativa di Ematologia - Ospedale S.,12,0,0,0,0,861 - Unità Operativa di Ematologia - Ospedale...
3,Calabria,587 - Presidio Morelli CTMO Centro Unico Regio...,111,41,15,17,9,587 - Presidio Morelli CTMO Centro Unico Regio...
4,Campania,191 - UOC Ematologia - ASL Salerno -Pagani,20,0,0,0,0,191 - UOC Ematologia - ASL Salerno -Pagani


In [15]:
len(result_df)

136

In [16]:
# Define the new order of columns and drop colum '1' with the beginnig of the original program name
new_column_order = ['0', 'Full Program Name', '1', '2', '3', '4', '5', '6']
df_reordered = result_df[new_column_order]
df_reordered= df_reordered.drop(labels=['1'], axis=1)
columns = ['region','full program name','autologous total','allogeneic total',
              'matched family donors','unrelated donors','haploidentical family donors']

In [17]:
df_reordered.columns = columns
df_reordered.head()

,region,full program name,autologous total,allogeneic total,matched family donors,unrelated donors,haploidentical family donors
0,Abruzzo,248 - UOSD Terapia Intensiva Ematologica -Dipa...,59,32,8,10,14
1,Basilicata,185 - UO Ematologia e Trapianto di Cellule Sta...,9,0,0,0,0
2,Basilicata,861 - Unità Operativa di Ematologia - Ospedale...,12,0,0,0,0
3,Calabria,587 - Presidio Morelli CTMO Centro Unico Regio...,111,41,15,17,9
4,Campania,191 - UOC Ematologia - ASL Salerno -Pagani,20,0,0,0,0


In [18]:
# converting numeric column in int dtype from str
for col in columns[2:]:
    df_reordered[col] = df_reordered[col].astype('int')

df_reordered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   region                        136 non-null    object
 1   full program name             136 non-null    object
 2   autologous total              136 non-null    int32 
 3   allogeneic total              136 non-null    int32 
 4   matched family donors         136 non-null    int32 
 5   unrelated donors              136 non-null    int32 
 6   haploidentical family donors  136 non-null    int32 
dtypes: int32(5), object(2)
memory usage: 4.9+ KB


In [19]:
df_reordered.tail()

,region,full program name,autologous total,allogeneic total,matched family donors,unrelated donors,haploidentical family donors
131,Veneto,"502.2 Dipartimento di Medicina, UOC Ematologia...",47,4,3,0,1
132,Veneto,623 - Unità di trapianto MO - PTC -Azienda Osp...,41,38,6,26,6
133,Veneto,623.1 CTMO Unità Adulti-Policlinico GB Rossi,35,34,6,24,4
134,Veneto,623.2 Oncoematologia Pediatrica/Azienda Ospeda...,6,4,0,2,2
135,Veneto,797 - Dipartimento di Ematologia - Presidio Os...,37,19,4,13,2


## Building the DataFrame for Data Visualization

I want to organize data in a table with the following structure:

| Program Name | Organ       | Number   | Region      | Year      | City        | Patient        |
|--------------|-------------|----------|-------------|-----------|-------------|----------------|
| Hospital XYZ |  HCT/Solid  | e.g. 70  | e.g. Veneto | e.g. 2023 | e.g. Verona | Adult/Pediatric|

This will allow to create a DataFrame with all names of transplant programs, number of tranplants performed each year per organ type, and geographical distribution.

To achieve this goal there are a few challenges to address starting from the current `df_reordered` DataFrame.
1. Infer whether the tranplant program is for *Adult*, *Pediatric*, or *Both* `Patient`s.
2. Extracting the city where the tranplant program is located from its `full program name` column.
3. Defining a common transplant program name for both Solid Organs and Hematopoietic Cells. In fact, although both types of data are provided by the **National Tranplant Center**, tranplants programs are named differently for Solid Organs and Hematopoietic Cells. This challenge needs some knowledge of the hospitals and making choices on which type of transplants giving priority for naming a program.

Because I want to focus only on allogeneic HCT programs, I will create a new DataFrame, excluding programs with only autologous tranplants.

In [20]:
df_allo = df_reordered[df_reordered['allogeneic total'] > 0].reset_index(drop=True)
df_allo.drop(labels='autologous total', axis=1, inplace=True) # dropping autologous column
df_allo.head()

,region,full program name,allogeneic total,matched family donors,unrelated donors,haploidentical family donors
0,Abruzzo,248 - UOSD Terapia Intensiva Ematologica -Dipa...,32,8,10,14
1,Calabria,587 - Presidio Morelli CTMO Centro Unico Regio...,41,15,17,9
2,Campania,341- Dipartimento Ematologia Pediatrica SSD TM...,18,2,10,6
3,Campania,607 - UOC di Ematologia con Trapianto di cell...,26,7,16,3
4,Campania,766- Ematologia e Trapianti di Midollo - AOU F...,17,4,9,4


Considering the relatively small number of transplant programs, and the many possible patterns in the program names, it seems easier to save the `df_allo` DataFrame as excel file, and execute the operations of
1. Inferring whether the tranplant program is *Adult*, *Pediatric*, or *Both*,
2. Extracting the city where the tranplant program is located from its `full program name` column,

directly into the excel file.

In [21]:
df_allo.to_excel('../data_cleaned/hct_data_2023.xlsx', index=False)

Here how the DataFrame looks after working on the `hct_data_2023.xlsx` file: 

In [22]:
file_path = '../data_cleaned/hct_data_2023_reworked.xlsx'
df_allo_reworked = pd.read_excel(file_path, sheet_name=0)

In [23]:
df_allo_reworked.head()

,region,full program name,allogeneic total,matched family donors,unrelated donors,haploidentical family donors,city,patient,Organ
0,Abruzzo,248 Ospedale Civile Santo Spirito,32,8,10,14,Pescara,adult,HCT
1,Calabria,587 Grande Ospedale Bianchi-Melacrino-Morelli,41,15,17,9,Reggio Calabria,adult,HCT
2,Campania,341 Azienda Ospedaliera Santobono Pausilipon,18,2,10,6,Napoli,pediatric,HCT
3,Campania,607 Ospedale A. Cardarelli,26,7,16,3,Napoli,adult,HCT
4,Campania,766 AOU Federico II,17,4,9,4,Napoli,adult,HCT


In [24]:
df_allo_reworked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   region                        56 non-null     object
 1   full program name             56 non-null     object
 2   allogeneic total              56 non-null     int64 
 3   matched family donors         56 non-null     int64 
 4   unrelated donors              56 non-null     int64 
 5   haploidentical family donors  56 non-null     int64 
 6   city                          56 non-null     object
 7   patient                       56 non-null     object
 8   Organ                         56 non-null     object
dtypes: int64(4), object(5)
memory usage: 4.1+ KB


This DataFrame needs further work once decided the common names to assign to programs for both Solid Organs and Hematopoietic Cells. After assigning the common names some details (e.g. type of patients, type of donors) can be removed to aggregate the HCT data to Solid Organ data.